In [1]:
import bcolz
import pickle
from tqdm import tqdm

In [2]:
import sys
sys.path.append("../")

In [3]:
glove_path = "./lib_data/glove.6B"

In [4]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(
    np.zeros(1), rootdir=f"{glove_path}/6B.50.dat", mode="w")

with open(f'{glove_path}/glove.6B.50d.txt', 'rb') as f:
    for l in tqdm(f):
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

vectors = bcolz.carray(vectors[1:].reshape((400001, 50)), rootdir=f'{glove_path}/6B.50.dat', mode='w')
vectors.flush()

400001it [00:28, 13939.70it/s]


In [5]:
pickle.dump(words, open(f"{glove_path}/6B.50_words.pkl", "wb"))
pickle.dump(word2idx, open(f"{glove_path}/6B.50_idx.pkl", "wb"))

In [6]:
vectors = bcolz.open(f"{glove_path}/6B.50.dat")[:]
words = pickle.load(open(f"{glove_path}/6B.50_words.pkl", "rb"))
word2idx = pickle.load(open(f"{glove_path}/6B.50_idx.pkl", "rb"))

glove = {w: vectors[word2idx[w]] for w in words}

In [7]:
vectors.shape

(400001, 50)

In [8]:
matrix_len = len(words)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0
for i, word in tqdm(enumerate(words)):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

400001it [00:00, 467304.97it/s]


In [10]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim